In [9]:
import pandas as pd

# --- File Paths ---
experimental_data_file = '../data/cleaned_experimental_data_combined.csv'
raw_data_file = '../data/rawdata.xlsx'
output_file = '../data/unique_ions_with_names.csv'

try:
    # --- 1. Load 'Table S3' without a header ---
    df_s3 = pd.read_excel(raw_data_file, sheet_name='Table S3', header=None)

    # --- 2. Extract lookup tables based on the correct column POSITIONS ---

    # Anion table is in columns 0 and 1
    anion_lookup = df_s3[[0, 1]].copy()
    anion_lookup.columns = ['ion_name', 'ion_abbreviation'] # Set the correct column names
    anion_lookup = anion_lookup.iloc[1:] # Skip the header row
    anion_lookup.dropna(inplace=True)
    anion_lookup['ion_type'] = 'anion'

    # Cation table is in columns 2 and 3
    cation_lookup = df_s3[[2, 3]].copy()
    cation_lookup.columns = ['ion_name', 'ion_abbreviation'] # Set the correct column names
    cation_lookup = cation_lookup.iloc[1:] # Skip the header row
    cation_lookup.dropna(inplace=True)
    cation_lookup['ion_type'] = 'cation'

    # Combine them into one master lookup table
    master_lookup = pd.concat([anion_lookup, cation_lookup], ignore_index=True)
    print("Successfully created a lookup table from 'Table S3'.")

    # --- 3. Find unique ion abbreviations from your main dataset ---
    df_exp = pd.read_csv(experimental_data_file)
    unique_cations = pd.DataFrame(df_exp['cation'].unique(), columns=['ion_abbreviation'])
    unique_anions = pd.DataFrame(df_exp['anion'].unique(), columns=['ion_abbreviation'])
    df_abbreviations = pd.concat([unique_cations, unique_anions], ignore_index=True).drop_duplicates()

    # --- 4. Merge the lists to find the full names ---
    df_final_list = pd.merge(df_abbreviations, master_lookup, on='ion_abbreviation', how='left')
    print(f"\nFound {len(df_final_list)} unique ions and their full names.")

    # --- 5. Save the final list ---
    df_final_list.to_csv(output_file, index=False)
    print(f"Final list with full names saved to '{output_file}'")

    print("\n--- Preview of Final Ion List ---")
    print(df_final_list.head())

except Exception as e:
    print(f"An error occurred: {e}")

Successfully created a lookup table from 'Table S3'.

Found 84 unique ions and their full names.
Final list with full names saved to '../data/unique_ions_with_names.csv'

--- Preview of Final Ion List ---
  ion_abbreviation                         ion_name ion_type
0       [(ETO)2IM]          1,3-diethoxyimidazolium   cation
1           [BBIM]           1,3-dibutylimidazolium   cation
2           [BMIM]     1-butyl-3-methyl-imidazolium   cation
3          [BMMIM]  1-butyl-2,3-dimethylimidazolium   cation
4          [BMPYR]   1-butyl-1-methyl-pyrrolidinium   cation
